In [7]:
import json
import os
import re
import fnmatch
from pathlib import Path
import numpy as np
import cv2
import copy
import random
import xml.etree.ElementTree as ET
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from pycocotools.coco import COCO

In [8]:
cls_ind_in_coco_set1 = [1, 2, 3, 4, 5, 6, 7, 9, 16, 17, 18, 19, 20, 21, 44, 62, 63, 64, 67, 72]
cls_ind_in_coco_set2 = [85, 43, 78, 87, 47, 11, 35, 53, 52, 46, 38, 51, 41, 58, 32, 82, 24, 37, 73, 22]
cls_ind_in_coco_set3 = [60, 65, 13, 48, 79, 77, 61, 54, 76, 34, 50, 74, 25, 86, 15, 31, 80, 14, 84, 28]
cls_ind_in_coco_set4 = [8, 10, 23, 27, 33, 36, 39, 40, 42, 49, 55, 56, 57, 59, 70, 75, 81, 88, 89, 90]

In [9]:
cwd = os.getcwd()
coco_json_path = cwd + '/data/coco/annotations/instances_train2014.json'
with open(coco_json_path, 'r') as f:
    data = json.load(f)

im_summary = {}
for i, im_d in tqdm(enumerate(data['images'])):
    im_summary[str(im_d['id'])] = {}
    im_summary[str(im_d['id'])]['im_dict'] = im_d
    im_summary[str(im_d['id'])]['annotations'] = []
    im_summary[str(im_d['id'])]['categories'] = []
for j, a_d in enumerate(data['annotations']):
    if a_d['iscrowd'] == 0:  # only keep non-crowd annotations
        im_id = a_d['image_id']
        cat_id = a_d['category_id']
        im_summary[str(im_id)]['annotations'].append(a_d)
        im_summary[str(im_id)]['categories'].append(cat_id)
for _key in im_summary.keys():
    im_summary[_key]['categories'] = list(set(im_summary[_key]['categories']))

In [10]:
novel_cls_inds = cls_ind_in_coco_set1
new_categories = []
for cat in data['categories']:
    if cat['id'] in novel_cls_inds:
        new_categories.append(cat)

MIN_SIZE = 16
cat_and_their_im_id = []
for i, c_d in enumerate(new_categories):
    cat_dict = {}
    cat_id = c_d['id']
    cat_dict['cat_id'] = cat_id
    cat_dict['im_ids'] = []
    for _key in im_summary.keys():  # for every im
        im_dict = im_summary[_key]
        if cat_id in im_dict['categories']:  # if this im has this class
            valid_an = []
            for an in im_dict['annotations']:  # for annos of this im
                if an['category_id'] == cat_id:
                    if an['bbox'][2] < MIN_SIZE or an['bbox'][3] < MIN_SIZE:
                        continue
                    else:
                        valid_an.append(an)
            if len(valid_an) > 0:  # only keep those id have more than one valid box of this class
                cat_dict['im_ids'].append(_key)           
    cat_and_their_im_id.append(cat_dict)
cat_and_their_im_id.sort(key=lambda s: len(s['im_ids']))

In [11]:
cwd = os.getcwd()
dump_dir = '/home/tony/datasets/coco/annotations/ft'
if not os.path.exists(dump_dir):
    os.makedirs(dump_dir)
IM_PER_CLASS = 10


### generate query json
id_have_selected = []
episode = []
for cat_dict in cat_and_their_im_id:
    new_dict = {}
    cat_id = cat_dict['cat_id']
    ids = cat_dict['im_ids']
    for _id in id_have_selected:  # remove im that has been picked
        if _id in ids:
            ids.remove(_id)
    random.seed(0)
    selected_ids = random.sample(ids, k=IM_PER_CLASS)
    id_have_selected.extend(selected_ids)
    new_dict['cat_id'] = cat_id
    new_dict['im_ids'] = selected_ids
    episode.append(new_dict)

new_data = {}
new_data['info'] = data['info']
new_data['images'] = []
new_data['licenses'] = data['licenses']
new_data['annotations'] = []
#     new_data['categories'] = new_categories
new_data['categories'] = data['categories']
for cat_dict in episode:
    cat_id = cat_dict['cat_id']
    id_list = cat_dict['im_ids']
    for _id in id_list:
        new_data['images'].append(im_summary[_id]['im_dict'])
        for an in im_summary[_id]['annotations']:
            if an['category_id'] == cat_id:
                new_data['annotations'].append(an)
dump_path = os.path.join(dump_dir, 'instances_shot10.json')
with open(dump_path, 'w') as f:
    json.dump(new_data, f)

In [12]:
cwd = os.getcwd()
dump_dir = '/home/tony/datasets/coco/annotations/ft'
if not os.path.exists(dump_dir):
    os.makedirs(dump_dir)
IM_PER_CLASS = 30


### generate query json
id_have_selected = []
episode = []
for cat_dict in cat_and_their_im_id:
    new_dict = {}
    cat_id = cat_dict['cat_id']
    ids = cat_dict['im_ids']
    for _id in id_have_selected:  # remove im that has been picked
        if _id in ids:
            ids.remove(_id)
    random.seed(0)
    selected_ids = random.sample(ids, k=IM_PER_CLASS)
    id_have_selected.extend(selected_ids)
    new_dict['cat_id'] = cat_id
    new_dict['im_ids'] = selected_ids
    episode.append(new_dict)

new_data = {}
new_data['info'] = data['info']
new_data['images'] = []
new_data['licenses'] = data['licenses']
new_data['annotations'] = []
#     new_data['categories'] = new_categories
new_data['categories'] = data['categories']
for cat_dict in episode:
    cat_id = cat_dict['cat_id']
    id_list = cat_dict['im_ids']
    for _id in id_list:
        new_data['images'].append(im_summary[_id]['im_dict'])
        for an in im_summary[_id]['annotations']:
            if an['category_id'] == cat_id:
                new_data['annotations'].append(an)
dump_path = os.path.join(dump_dir, 'instances_shot30.json')
with open(dump_path, 'w') as f:
    json.dump(new_data, f)